In [1]:
import os
os.environ['GRB_LICENSE_FILE'] = "C:/Users/agust/gurobi.lic"

import random
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
import gurobipy as gp

In [2]:
# Create World Cup rankings based on official FIFA ranking formulae

# Assign datasets
matches = pd.read_csv("C:/Users/agust/Downloads/FIFA World Cup 1930-2022 All Match Dataset.csv", encoding='ISO-8859-1')
confederations = pd.read_csv("C:/Users/agust/Downloads/confederations.csv", encoding='ISO-8859-1')

# Convert 'Match Date' to datetime and extract year
matches['Match Date'] = pd.to_datetime(matches['Match Date'])
matches['Year'] = matches['Match Date'].dt.year

# Initialize points for each team based on initial appearance
teams = pd.unique(matches[['Home Team Name', 'Away Team Name']].values.ravel('K'))
team_points = {team: 1000 for team in teams}  # Starting points can be adjusted as necessary

# Define match importance based on type of match
importance_dict = {
    'world cup group': 1,
    'world cup knockout': 2
}

# Determine range of years for linear scaling
min_year = matches['Year'].min()
max_year = matches['Year'].max()
year_range = max_year - min_year

# Function to calculate linear year weight
def get_year_weight(year):
    return 1 + ((year - min_year) / year_range)  # Scales from 1.0 to 2.0

# Function to calculate expected result
def calculate_expected_result(pa, pb):
    dr = pa - pb
    return 1 / (10 ** (-dr / 600) + 1)

# Process each match and update points
for _, match in matches.iterrows():
    home_team = match['Home Team Name']
    away_team = match['Away Team Name']
    year = match['Year']
    year_weight = get_year_weight(year)  # Get the weight for the year of the match
    
    # Calculate match importance adjusted by year
    base_importance = importance_dict.get(match['Stage Name'].lower(), 10)
    adjusted_importance = base_importance * year_weight  # Adjusting importance by year weight

    knockout_stage = match['Knockout Stage'] == 1
    penalty_shootout = match['Penalty Shootout'] == 1

    # Determine results with rules for penalty shootouts
    if penalty_shootout:
        if match['Home Team Win']:
            result_home = 0.75
            result_away = 0.5
        else:
            result_home = 0.5
            result_away = 0.75
    else:
        if match['Home Team Win']:
            result_home = 1
            result_away = 0
        elif match['Away Team Win']:
            result_home = 0
            result_away = 1
        else:
            result_home = 0.5
            result_away = 0.5

    p_before_home = team_points[home_team]
    p_before_away = team_points[away_team]

    expected_home = calculate_expected_result(p_before_home, p_before_away)
    expected_away = calculate_expected_result(p_before_away, p_before_home)

    change_home = adjusted_importance * (result_home - expected_home)
    change_away = adjusted_importance * (result_away - expected_away)

    # Apply knock-out round rule to avoid point loss
    if knockout_stage:
        if change_home < 0:
            change_home = 0
        if change_away < 0:
            change_away = 0

    team_points[home_team] += change_home
    team_points[away_team] += change_away

# Filter out extinct countries
non_existent_countries = ['Yugoslavia', 'Czechoslovakia', 'West Germany', 'Soviet Union', 'East Germany', 'Dutch East Indies', 'Zaire']
team_points = {team: points for team, points in team_points.items() if team not in non_existent_countries}

In [3]:
teams_data = pd.DataFrame(list(team_points.items()), columns=['Country', 'Points'])

# Merge with the confederations data to get confederation information
rankings_df = teams_data.merge(confederations, how='left', on='Country')

In [4]:
# Define the host nations
host_nations = ['Canada', 'Mexico', 'United States']

# Initialize the list of qualified teams with host nations
qualified_teams = host_nations.copy()

# Define the qualification slots for each confederation
qualification_slots = {
    'CONCACAF': 3,  # Not including the host nations
    'AFC': 8,
    'CAF': 9,
    'CONMEBOL': 6,
    'OFC': 1,
    'UEFA': 16,
    'Playoff': 2  # Top 2 teams not already qualified
}

# Function to select top teams based on the number of slots
def select_top_teams(df, conf, slots):
    conf_teams = df[df['Confederation'] == conf]
    return conf_teams.nlargest(slots, 'Points')['Country'].tolist()

# Select top teams from each confederation
for conf, slots in qualification_slots.items():
    if conf == 'Playoff':
        continue
    conf_slots = slots
    if conf == 'CONCACAF':
        # Exclude host nations from CONCACAF qualification
        concacaf_teams = rankings_df[(rankings_df['Confederation'] == conf) & (~rankings_df['Country'].isin(host_nations))]
        qualified_teams.extend(concacaf_teams.nlargest(conf_slots, 'Points')['Country'].tolist())
    else:
        qualified_teams.extend(select_top_teams(rankings_df, conf, conf_slots))

# Select playoff teams: top 2 teams not already qualified
remaining_teams = rankings_df[~rankings_df['Country'].isin(qualified_teams)]
playoff_teams = remaining_teams.nlargest(qualification_slots['Playoff'], 'Points')['Country'].tolist()
qualified_teams.extend(playoff_teams)

# Verify the number of qualified teams
num_qualified_teams = len(qualified_teams)
qualified_teams, num_qualified_teams

(['Canada',
  'Mexico',
  'United States',
  'Cuba',
  'Costa Rica',
  'Jamaica',
  'Japan',
  'Kuwait',
  'North Korea',
  'United Arab Emirates',
  'China',
  'Iraq',
  'Qatar',
  'Australia',
  'Senegal',
  'Morocco',
  'Ghana',
  'South Africa',
  'Angola',
  'Ivory Coast',
  'Nigeria',
  'Algeria',
  'Togo',
  'Brazil',
  'Argentina',
  'Uruguay',
  'Colombia',
  'Paraguay',
  'Chile',
  'New Zealand',
  'Netherlands',
  'France',
  'Germany',
  'Italy',
  'England',
  'Spain',
  'Belgium',
  'Croatia',
  'Portugal',
  'Sweden',
  'Switzerland',
  'Poland',
  'Romania',
  'Denmark',
  'Turkey',
  'Hungary',
  'Republic of Ireland',
  'Ukraine'],
 48)

In [19]:
# Initialize Gurobi model for group stage optimization
group_model = Model('WorldCupGroupStage')

# Define parameters
num_groups = 12
teams_per_group = 4
num_teams = len(qualified_teams)

# Group labels from 'A' to 'L'
group_labels = list('ABCDEFGHIJKL')

# Create a dictionary for confederation of each qualified team
team_conf = {team: rankings_df[rankings_df['Country'] == team]['Confederation'].values[0] for team in qualified_teams}
team_points = {team: rankings_df[rankings_df['Country'] == team]['Points'].values[0] for team in qualified_teams}

# Decision variables: x[i,j] = 1 if team i is in group j, else 0
x = group_model.addVars(qualified_teams, range(num_groups), vtype=GRB.BINARY, name="x")

# Auxiliary binary variables to indicate if a confederation is represented in a group
conf_rep = group_model.addVars(qualification_slots.keys(), range(num_groups), vtype=GRB.BINARY, name="conf_rep")

# Constraints
# Each team must be assigned to exactly one group
for team in qualified_teams:
    group_model.addConstr(quicksum(x[team, j] for j in range(num_groups)) == 1, name=f"Team_{team}_Assignment")

# Each group must have exactly 4 teams
for j in range(num_groups):
    group_model.addConstr(quicksum(x[team, j] for team in qualified_teams) == teams_per_group, name=f"Group_{j}_Size")

# Ensure at least 3 different confederations per group
for j in range(num_groups):
    for conf in qualification_slots.keys():
        if conf == 'Playoff':
            continue
        conf_teams = [team for team in qualified_teams if team_conf[team] == conf]
        group_model.addConstr(quicksum(x[team, j] for team in conf_teams) >= conf_rep[conf, j], name=f"Group_{j}_Conf_{conf}_MinRep")
        group_model.addConstr(conf_rep[conf, j] <= 1, name=f"Group_{j}_Conf_{conf}_MaxRep")

    # Ensure at least 3 different confederations per group
    group_model.addConstr(quicksum(conf_rep[conf, j] for conf in qualification_slots.keys() if conf != 'Playoff') >= 3, name=f"Group_{j}_MinConfederations")

# Balance the groups based on team points
avg_group_points = sum(team_points.values()) / num_groups
max_group_diff = 1000

# Add constraints for balancing group points
for j in range(num_groups):
    group_model.addConstr(quicksum(x[team, j] * team_points[team] for team in qualified_teams) <= avg_group_points + max_group_diff, name=f"Group_{j}_MaxPoints")
    group_model.addConstr(quicksum(x[team, j] * team_points[team] for team in qualified_teams) >= avg_group_points - max_group_diff, name=f"Group_{j}_MinPoints")

# Objective to minimize the maximum deviation of group points from the average
group_points = [quicksum(x[team, j] * team_points[team] for team in qualified_teams) for j in range(num_groups)]
max_deviation = group_model.addVar(vtype=GRB.CONTINUOUS, name="max_deviation")

for j in range(num_groups):
    group_model.addConstr(group_points[j] <= avg_group_points + max_deviation, name=f"Group_{j}_MaxDeviation")
    group_model.addConstr(group_points[j] >= avg_group_points - max_deviation, name=f"Group_{j}_MinDeviation")

group_model.setObjective(max_deviation, GRB.MINIMIZE)

# Optimize the model
group_model.optimize()

# Extract the group assignments
groups = {label: [] for label in group_labels}
if group_model.status == GRB.OPTIMAL:
    for j, label in enumerate(group_labels):
        for team in qualified_teams:
            if x[team, j].x > 0.5:
                groups[label].append(team)

groups

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 4500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Academic license 2525116 - for non-commercial use only - registered to aa___@scu.edu
Optimize a model with 264 rows, 661 columns and 4272 nonzeros
Model fingerprint: 0x1b668423
Variable types: 1 continuous, 660 integer (660 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 187.8423128
Presolve removed 108 rows and 24 columns
Presolve time: 0.00s
Presolved: 156 rows, 637 columns, 3000 nonzeros
Variable types: 1 continuous, 636 integer (636 binary)

Root relaxation: objective 5.447372e-01, 443 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      

{'A': ['China', 'Uruguay', 'Sweden', 'Republic of Ireland'],
 'B': ['Mexico', 'Nigeria', 'Croatia', 'Ukraine'],
 'C': ['South Africa', 'Colombia', 'Chile', 'Portugal'],
 'D': ['United Arab Emirates', 'Angola', 'Italy', 'Romania'],
 'E': ['United States', 'Ghana', 'Germany', 'Poland'],
 'F': ['Iraq', 'Algeria', 'Argentina', 'Switzerland'],
 'G': ['Japan', 'Senegal', 'Paraguay', 'Belgium'],
 'H': ['North Korea', 'New Zealand', 'Netherlands', 'Hungary'],
 'I': ['Jamaica', 'Qatar', 'Ivory Coast', 'France'],
 'J': ['Cuba', 'Kuwait', 'Spain', 'Denmark'],
 'K': ['Costa Rica', 'Morocco', 'England', 'Turkey'],
 'L': ['Canada', 'Australia', 'Togo', 'Brazil']}